In [1]:
!git clone https://github.com/Muennighoff/performers.git
!cd performers; pip install -q -e '.[dev]'

!pip install -q datasets
!pip install -q ml_collections
!pip install -q --upgrade jax jaxlib
!pip install -q ltp

Cloning into 'performers'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 56020 (delta 0), reused 2 (delta 0), pack-reused 56013
Receiving objects: 100% (56020/56020), 42.39 MiB | 22.57 MiB/s, done.
Resolving deltas: 100% (39265/39265), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 6.3MB/s 
     |████████████████████████████████| 2.9MB 16.2MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 1.1MB 51.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 31.9MB 263kB/s 
     |████████████████████████████████| 13.4MB 50.1MB/s 
     |████████████████████████████████| 163kB 29.7MB/s 
     |██████████

In [10]:
import os
import sys
import tensorflow as tf
import math
from datasets import load_dataset

module_path = os.path.abspath(os.path.join('./performers/src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from transformers.models.distilbert import DistilBertConfig, DistilBertForSequenceClassification
from transformers.models.distilbert import TFDistilBertForSequenceClassification, TFDistilBertForMaskedLM
from transformers.models.bert import BertConfig, TFBertForSequenceClassification

In [6]:
# If this cell takes very long to run; factory reset and run this cell the very first

tf.keras.backend.clear_session()

try:
    # Locate TPU
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    # Establish communication / Locates TPU on the network
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)
    # Connect
    tf.config.experimental_connect_to_cluster(resolver)
    # Start TPU
    tf.tpu.experimental.initialize_tpu_system(resolver)
    # Set strategy
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

    print("Number of accelerators: ", strategy.num_replicas_in_sync)

except ValueError:
    print("No TPU found")

# Check devices
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.5.69.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.5.69.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8
All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


##### Tokenizer

In [2]:
## Whole Word Masking

### WWM for CN

# References:
# https://github.com/brightmart/roberta_zh/blob/13f7849f0cb0e11573e032acddb35b83b096224e/create_pretraining_data.py
# https://github.com/huggingface/transformers/blob/9152f16023b59d262b51573714b40325c8e49370/examples/legacy/run_chinese_ref.py#L78


# 1) Generate ref ids based on LTP tokenizer > prepare_ref
# 2) Generate mask for whole words
# 3) Implement the masking

from transformers import AutoTokenizer
import random
import numpy as np
from ltp import LTP

import tensorflow as tf


def random_word(tokens, ref_ids, tokenizer):
    """
    Masking some random tokens for Language Model task with probabilities as in the original BERT paper.
    :param tokens: list of str, tokenized sentence.
    :param ref_ids: list of int, 1 is where to place a mask
    :param tokenizer: Tokenizer, object used for tokenization (we need it's vocab here)
    :return: (list of str, list of int), masked tokens and related labels for LM prediction
    """
    output_label = []

    for i, (token, ref_id) in enumerate(zip(tokens, ref_ids)):

        prob = random.random()

        if ref_id == 1:

            # 80% randomly change token to mask token
            if prob < 0.8:
                tokens[i] = "[MASK]"

            # 10% randomly change token to random token
            elif prob < 0.9:
                tokens[i] = random.choice(list(tokenizer.vocab.items()))[0]

            # -> rest 10% randomly keep current token

            # append current token to output (we will predict these later)
            try:
                output_label.append(tokenizer.vocab[token])
            except KeyError:
                # For unknown words (should not occur with BPE vocab)
                output_label.append(tokenizer.vocab["[UNK]"])
        else:
            # no masking token (will be ignored by loss function later)
            output_label.append(-100)

    return tokens, output_label

def _is_chinese_char(cp):
    """
    Checks whether CP is the codepoint of a CJK character.
    """
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False


def is_chinese(word):
    """
    Args:
      word: str
    """
    # word like '180' or '身高' or '神'
    for char in word:
        char = ord(char)
        if not _is_chinese_char(char):
            return 0
    return 1


def get_chinese_word(tokens):
    """
    Args:
      List[str]
    
    """
    word_set = set()

    for token in tokens:
        chinese_word = len(token) > 1 and is_chinese(token)
        if chinese_word:
            word_set.add(token)
    word_list = list(word_set)
    return word_list


def add_sub_symbol(bert_tokens, chinese_word_set):
    """
    Args:
      bert_tokens: List[str]
      chinese_word_set: set
    """
    if not chinese_word_set:
        return bert_tokens
    max_word_len = max([len(w) for w in chinese_word_set])

    bert_word = bert_tokens
    start, end = 0, len(bert_word)
    while start < end:
        single_word = True
        if is_chinese(bert_word[start]):
            l = min(end - start, max_word_len)
            for i in range(l, 1, -1):
                whole_word = "".join(bert_word[start : start + i])
                if whole_word in chinese_word_set:
                    for j in range(start + 1, start + i):
                        bert_word[j] = "##" + bert_word[j]
                    start = start + i
                    single_word = False
                    break
        if single_word:
            start += 1
    return bert_word

def prepare_ref(lines, ltp_tokenizer, bert_tokenizer):
    """
    
    Args:
      lines: List[str] - e.g. [text1, text2]
      ltp_tokenizer
      bert_tokenizer

    Returns:
      ref_ids: List[List[int], ...]
    
    """

    ltp_res = []

    for i in range(0, len(lines), 100):
        res = ltp_tokenizer.seg(lines[i : i + 100])[0]
        res = [get_chinese_word(r) for r in res]
        ltp_res.extend(res)
    assert len(ltp_res) == len(lines)

    bert_res = []
    for i in range(0, len(lines), 100):
        res = bert_tokenizer(lines[i : i + 100], add_special_tokens=True, truncation=True, max_length=512)
        bert_res.extend(res["input_ids"])
    assert len(bert_res) == len(lines)

    ref_ids = []
    for input_ids, chinese_word in zip(bert_res, ltp_res):

        input_tokens = []
        for id in input_ids:
            token = bert_tokenizer._convert_id_to_token(id)
            input_tokens.append(token)
        input_tokens = add_sub_symbol(input_tokens, chinese_word)
        ref_id = []
        # We only save pos of chinese subwords start with ##, which mean is part of a whole word.
        for i, token in enumerate(input_tokens):
            if token[:2] == "##":
                clean_token = token[2:]
                # save chinese tokens' pos
                if len(clean_token) == 1 and _is_chinese_char(ord(clean_token)):
                    ref_id.append(i)
        ref_ids.append(ref_id)

    assert len(ref_ids) == len(bert_res)

    return ref_ids


def cn_whole_word_mask(input_tokens, ref_ids, max_predictions=512, mlm_probability=0.15):
    """
    Masks whole words in CN based on the reference ids & the standard _whole_word_mask for BERT for one individual example.

    Args:
      input_tokens: List[str]
      ref_tokens: List[int]

    Returns:
      input_tokens: List[int]

    TODO:
      We could save the LTP dependency by copying the function from: https://github.com/HIT-SCIR/ltp/blob/c47b3f455c07c5dcc186f2b674efde8c67612baf/ltp/algorithms/maximum_forward_matching.py#L75
    """

    for i in range(len(input_tokens)):
        if i in ref_ids:
            # We move it back by -1 as the ref_ids start at 1, not 0
            input_tokens[i-1] = "##" + input_tokens[i-1]

    input_tokens = _whole_word_mask(input_tokens)

    return input_tokens



def _whole_word_mask(input_tokens, max_predictions=512, mlm_probability=0.15):
    """
    Get 0/1 labels for masked tokens with whole word mask proxy

    Args:
      input_tokens: List[str]

    Outputs:
      input_tokens: List[int]
    """

    cand_indexes = []
    for (i, token) in enumerate(input_tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue

        if len(cand_indexes) >= 1 and token.startswith("##"):
            cand_indexes[-1].append(i)

        else:
            cand_indexes.append([i])

    random.shuffle(cand_indexes)
    num_to_predict = min(max_predictions, max(1, int(round(len(input_tokens) * mlm_probability))))
    masked_lms = []
    covered_indexes = set()
    for index_set in cand_indexes:
        if len(masked_lms) >= num_to_predict:
            break
        # If adding a whole-word mask would exceed the maximum number of
        # predictions, then just skip this candidate.
        if len(masked_lms) + len(index_set) > num_to_predict:
            continue
        is_any_index_covered = False
        for index in index_set:
            if index in covered_indexes:
                is_any_index_covered = True
                break
        if is_any_index_covered:
            continue
        for index in index_set:
            covered_indexes.add(index)
            masked_lms.append(index)

    assert len(covered_indexes) == len(masked_lms)
    mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_tokens))]
    return mask_labels



class WWMTokenizer():
    def __init__(self, col="text", seq_len=512):
        """
        Constructs Huggingface CN tokenizer & other

            col: What column to tokenize if pretraining
        """

        self.tokenizer_cn = AutoTokenizer.from_pretrained("bert-base-chinese")
        self.tokenizer_ltp = LTP("small")
        self.max_seq_length = seq_len
        self.col = col

    def tokenize_pretraining(self, example):
        """
        Takes in an example & returns pretraining data

        Args:
            Example: dict with entry "text"

        Returns:
            Dict of TF Tensors

        """

        inputs = example[self.col]
    

        ref_ids = prepare_ref([inputs], self.tokenizer_ltp, self.tokenizer_cn)

        tokens = self.tokenizer_cn.tokenize(inputs)

        if len(tokens) > self.max_seq_length - 2:
            tokens = tokens[:(self.max_seq_length - 2)]
            ref_ids = ref_ids[:(self.max_seq_length - 2)]

        ref_ids = cn_whole_word_mask(tokens, ref_ids[0])
        tokens, labels = random_word(tokens, ref_ids, self.tokenizer_cn)

        tokens = ['[CLS]'] + tokens + ['[SEP]']
        lm_label_ids = ([-100] + labels + [-100])

        input_ids = self.tokenizer_cn.convert_tokens_to_ids(tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            attention_mask.append(0)
            token_type_ids.append(0)
            lm_label_ids.append(-100)

        assert len(input_ids) == self.max_seq_length
        assert len(attention_mask) == self.max_seq_length
        assert len(token_type_ids) == self.max_seq_length
        assert len(lm_label_ids) == self.max_seq_length


        outputs = {'input_ids': tf.constant(input_ids), 'attention_mask': tf.constant(attention_mask), 
                'token_type_ids': tf.constant(token_type_ids), 'lm_label_ids': tf.constant(lm_label_ids)}

        return outputs

    def tokenize_cmnli(self, example):
        """
        Tokenizes data for the CMNLI dataset

        Call this with the filter method to sort out datapoints with an unclean label of '-'
        """

        # > [CLS] sentence1 [SEP] sentence2 [SEP]
        inputs = self.tokenizer_cn(text=example["sentence1"], text_pair=example["sentence2"], 
                                   padding="max_length", max_length=self.max_seq_length)

        # SparseCategoricalCrossEntropy expects floating points
        label_map = {
            "neutral":0.0,
            "entailment":1.0,
            "contradiction":2.0,
            "-": 3.0,              # Some unclean datapoints in the val dataset; We remove them in the next step
        }

        label = label_map[example["label"]]

        return {'input_ids': inputs["input_ids"], 'token_type_ids': inputs["token_type_ids"], 'attention_mask': inputs["attention_mask"], 'label': label}

    def to_tf_dataset(self, dataset): 
        """
        Turns dataset into a TF compatible dataset
        """
        columns = ['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids']
        dataset.set_format(type='tensorflow', columns=columns)

        return_types = {'input_ids':tf.int32, 'attention_mask':tf.int32, 
                      'token_type_ids':tf.int32, 'lm_label_ids':tf.int32}

        ds = tf.data.Dataset.from_generator(lambda : dataset, return_types)
        return ds

##### Pretraining - Data

In [3]:
### Prep Functions ###

### Modelling - Static ###

class Wrapper(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        # Track loss (Loss itself its CategoricalCrossEnt.)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss') 

    @tf.function
    def train_step(self, data):
        # Data is a dictionary
        y = data["lm_label_ids"]
        x = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.model(x, training=True)
            # Compute the loss value.
            y_pred = y_pred.logits
            loss = self.compute_loss(y, y_pred)
            # Reduce loss to single digit
            loss = tf.reduce_mean(loss)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update loss tracker
        self.loss_tracker.update_state(loss)  
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        # Data is a dictionary
        y = data["lm_label_ids"]
        x = data

        # Compute predictions
        y_pred = self.model(x, training=False)
        y_pred = y_pred.logits
        loss = self.compute_loss(y, y_pred)

        # Updates the metrics tracking the loss
        self.loss_tracker.update_state(loss)  
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def compute_loss(self, labels, logits):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE
        )
        # make sure only labels that are not equal to -100 do affect loss
        active_loss = tf.not_equal(tf.reshape(labels, (-1,)), -100)
        reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
        labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

        return loss_fn(labels, reduced_logits)
        
def shape_list(tensor: tf.Tensor):
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (:obj:`tf.Tensor`): The tensor we want the shape of.
    Returns:
        :obj:`List[int]`: The shape of the tensor as a list.

    ### TODO: Move to compute_loss
    """
    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]


def tf_dataset_slices(train_ds, tokenizer):
    """
    Diff method of turning into TF Dataset
    """
    train_ds.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids'])
    features = {x: train_ds[x].to_tensor(default_value=0, shape=[None, tokenizer.tokenizer_cn.model_max_length]) for x in ['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids']}
    train_ds = tf.data.Dataset.from_tensor_slices(features)

    return train_ds


def get_dataset(batch_size, seq_len=512, is_training=True):
    """
    Only train for now;
    """
    train_ds = load_dataset('c3', 'mixed', split='train')
    val_ds = load_dataset('c3', 'mixed', split='validation')

    def prepare_dict(example):
        example["text"] = example["documents"][0]
        return example
    train_ds = train_ds.map(prepare_dict)
    val_ds = val_ds.map(prepare_dict)

    print(len(train_ds), len(val_ds))

    # Shorten for testing
    #train_ds = train_ds.select(range(32))
    #val_ds = val_ds.select(range(32))

    tokenizer = WWMTokenizer(col="text", seq_len=seq_len)

    train_ds = train_ds.map(tokenizer.tokenize_pretraining)
    val_ds = val_ds.map(tokenizer.tokenize_pretraining)

    train_ds = tf_dataset_slices(train_ds, tokenizer)
    val_ds = tf_dataset_slices(val_ds, tokenizer)

    # Need to drop the remainder if len(ds) // bs != 0
    train_ds = train_ds.shuffle(1000).batch(batch_size, drop_remainder=True)
    val_ds = val_ds.batch(batch_size, drop_remainder=True)

    return train_ds, val_ds

##### Finetuning - Data

In [6]:
# Finetuning dataset
!wget https://storage.googleapis.com/cluebenchmark/tasks/cmnli_public.zip
!unzip /content/cmnli_public.zip

--2021-02-08 16:36:07--  https://storage.googleapis.com/cluebenchmark/tasks/cmnli_public.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31404066 (30M) [application/zip]
Saving to: ‘cmnli_public.zip’

cmnli_public.zip    100%[===================>]  29.95M  72.1MB/s    in 0.4s    

2021-02-08 16:36:07 (72.1 MB/s) - ‘cmnli_public.zip’ saved [31404066/31404066]

Archive:  /content/cmnli_public.zip
   creating: cmnli_public/
  inflating: cmnli_public/test.json  
  inflating: cmnli_public/dev.json   
  inflating: cmnli_public/train.json  


In [17]:
### Prep Functions ###

class WrapperFinetuning(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        # Track loss (Loss itself its CategoricalCrossEnt.)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss') 

    @tf.function
    def train_step(self, data):
        # Data is a tuple of x, y for the CMNLI Finetuning data
        x, y = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.model(x, training=True)
            # Compute the loss value.
            y_pred = y_pred.logits
            loss = self.compute_loss(y, y_pred)
            # Reduce loss to single digit
            loss = tf.reduce_mean(loss)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update loss tracker
        self.loss_tracker.update_state(loss)  
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        x, y = data

        # Compute predictions
        y_pred = self.model(x, training=False)
        y_pred = y_pred.logits
        loss = self.compute_loss(y, y_pred)

        # Updates the metrics tracking the loss
        self.loss_tracker.update_state(loss)  
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def compute_loss(self, labels, logits):
        if len(shape_list(logits)) == 1 or shape_list(logits)[1] == 1:
            loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
        else:
            # We use the below, as 3 labels
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True, reduction=tf.keras.losses.Reduction.NONE
            )

        return loss_fn(labels, logits)


def shape_list(tensor: tf.Tensor):
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (:obj:`tf.Tensor`): The tensor we want the shape of.
    Returns:
        :obj:`List[int]`: The shape of the tensor as a list.
    """
    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]


def to_tf_cmnli(ds, tokenizer):
    """
    Creates a TF Dataset for the CMNLI dataset
    """

    # Turn into TF Tensors
    ds.set_format('tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

    # Extract TF Tensors
    features = {x: ds[x].to_tensor(default_value=0, shape=[None, tokenizer.tokenizer_cn.model_max_length]) for x in ['input_ids', 'token_type_ids', 'attention_mask']}

    # Label is not a ragged tensor like the others & hence we separate it 
    tfds = tf.data.Dataset.from_tensor_slices((features, ds["label"]))

    return tfds


def get_dataset_f(batch_size=8, seq_len=512, is_training=True):
    """
    Only train & val for now
    """
    train_ds_f = load_dataset("json", data_files="/content/cmnli_public/train.json", split="train")
    val_ds_f = load_dataset("json", data_files="/content/cmnli_public/dev.json", split="train")
    #test_ds_f = load_dataset("json", data_files="/content/cmnli_public/test.json", split="train")

    # Shorten for testing
    train_ds_f = train_ds_f.select(range(len(train_ds_f)//4))
    val_ds_f = val_ds_f.select(range(len(val_ds_f)//4))

    tokenizer = WWMTokenizer(col="text", seq_len=seq_len)

    train_ds_f = train_ds_f.map(tokenizer.tokenize_cmnli)
    val_ds_f = val_ds_f.map(tokenizer.tokenize_cmnli)

    def clean_data(example):
        """
        Clean up misplaced labels in val dataset
        """
        if example["label"] == 3.0:
            return False
        return True

    val_ds_f = val_ds_f.filter(clean_data)

    train_ds_f = to_tf_cmnli(train_ds_f, tokenizer)
    val_ds_f = to_tf_cmnli(val_ds_f, tokenizer)

    # Need to drop the remainder if len(ds) // bs != 0
    train_ds_f = train_ds_f.shuffle(1000).batch(batch_size, drop_remainder=True)
    val_ds_f = val_ds_f.batch(batch_size, drop_remainder=True)

    return train_ds_f, val_ds_f

def get_dataset_f_pre(batch_size=8, seq_len=512, is_training=True):
    """
    Pre-train on the finetuning dataset
    """
    train_ds = load_dataset("json", data_files="/content/cmnli_public/train.json", split="train")
    val_ds = load_dataset("json", data_files="/content/cmnli_public/dev.json", split="train")

    def prepare_dict(example):
        example["text"] = example["sentence1"] + " " + example["sentence2"]
        return example
    train_ds = train_ds.map(prepare_dict)
    val_ds = val_ds.map(prepare_dict)

    print(len(train_ds), len(val_ds))

    # Shorten for testing
    train_ds = train_ds.select(range(len(train_ds)//4))
    val_ds = val_ds.select(range(len(val_ds)//4))

    tokenizer = WWMTokenizer(col="text", seq_len=seq_len)

    train_ds = train_ds.map(tokenizer.tokenize_pretraining)
    val_ds = val_ds.map(tokenizer.tokenize_pretraining)

    train_ds = tf_dataset_slices(train_ds, tokenizer)
    val_ds = tf_dataset_slices(val_ds, tokenizer)

    # Need to drop the remainder if len(ds) // bs != 0
    train_ds = train_ds.shuffle(1000).batch(batch_size, drop_remainder=True)
    val_ds = val_ds.batch(batch_size, drop_remainder=True)

    return train_ds, val_ds

##### Finetuning from scratch - Performer close to on par with Transformer

In [7]:
### Finetune - Performer

with strategy.scope():
    config = DistilBertConfig()
    config.attention_type = "performer"
    # Necessary for the classification head created in the model, as CMNLI has 3 labels
    config.num_labels = 3
    config.vocab_size = 21128 # BERT CN Vocab only has 21128
    config.max_position_embeddings = 512 # same as seq_len

    model = TFDistilBertForSequenceClassification(config)
    model = WrapperFinetuning(model)

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  steps_per_execution=100,
                  metrics=['sparse_categorical_accuracy'])

batch_size = 8

train_ds, val_ds = get_dataset_f(batch_size, seq_len=512, is_training=True)

model.fit(
          train_ds,
          epochs=5,
          validation_data=val_ds,
          )

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-a4cf23710025995b/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-92b47242d2dd7ae8/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)



Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f3a307ea2a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f3a13f4d798>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f3a307ea2a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f3a307ea2a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.4124 - loss: 1.0492

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - 296s 24ms/step - sparse_categorical_accuracy: 0.4124 - loss: 1.0492 - val_sparse_categorical_accuracy: 0.4590 - val_loss: 1.0243
Epoch 2/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.4766 - loss: 1.0036 - val_sparse_categorical_accuracy: 0.4685 - val_loss: 1.0064
Epoch 3/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.5017 - loss: 0.9740 - val_sparse_categorical_accuracy: 0.4678 - val_loss: 1.0134
Epoch 4/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.5294 - loss: 0.9416 - val_sparse_categorical_accuracy: 0.5033 - val_loss: 1.0066
Epoch 5/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.5580 - loss: 0.9018 - val_sparse_categorical_accuracy: 0.5157 - val_loss: 1.0097


In [19]:
### Finetune - Traditional Transformer

with strategy.scope():
    config = DistilBertConfig()
    config.attention_type = "softmax"
    # Necessary for the classification head created in the model, as CMNLI has 3 labels
    config.num_labels = 3
    config.vocab_size = 21128 # BERT CN Vocab only has 21128
    config.max_position_embeddings = 512 # same as seq_len

    model = TFDistilBertForSequenceClassification(config)
    model = WrapperFinetuning(model)

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  steps_per_execution=100,
                  metrics=['sparse_categorical_accuracy'])

batch_size = 8

train_ds, val_ds = get_dataset_f(batch_size, seq_len=512, is_training=True)

model.fit(
          train_ds,
          epochs=5,
          validation_data=val_ds,
          )

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-a4cf23710025995b/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-92b47242d2dd7ae8/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.4151 - loss: 1.0453

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - 284s 23ms/step - sparse_categorical_accuracy: 0.4151 - loss: 1.0453 - val_sparse_categorical_accuracy: 0.4646 - val_loss: 1.0346
Epoch 2/5
12243/12243 [==============================] - 247s 20ms/step - sparse_categorical_accuracy: 0.4899 - loss: 0.9930 - val_sparse_categorical_accuracy: 0.4882 - val_loss: 0.9960
Epoch 3/5
12243/12243 [==============================] - 247s 20ms/step - sparse_categorical_accuracy: 0.5171 - loss: 0.9617 - val_sparse_categorical_accuracy: 0.5030 - val_loss: 0.9839
Epoch 4/5
12243/12243 [==============================] - 247s 20ms/step - sparse_categorical_accuracy: 0.5403 - loss: 0.9324 - val_sparse_categorical_accuracy: 0.5161 - val_loss: 0.9658
Epoch 5/5
12243/12243 [==============================] - 247s 20ms/step - sparse_categorical_accuracy: 0.5649 - loss: 0.9010 - val_sparse_categorical_accuracy: 0.5233 - val_loss: 0.9726


##### Pretrain then Finetune - Works well; No performance boost for short duration

In [13]:
### Pretrain


with strategy.scope():
    config = DistilBertConfig()
    config.attention_type = "performer"
    config.vocab_size = 21128 # BERT CN Vocab only has 21128
    config.max_position_embeddings = 512 # same as seq_len

    model = TFDistilBertForMaskedLM(config)
    model = Wrapper(model)

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  steps_per_execution=100,
                  metrics=['sparse_categorical_accuracy'])

batch_size = 8

train_ds, val_ds = get_dataset(batch_size, seq_len=512, is_training=True)

model.fit(
          train_ds,
          epochs=5,
          validation_data=val_ds,
          )

Reusing dataset c3 (/root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c)
Reusing dataset c3 (/root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c/cache-d5d515a16fa8e30f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c/cache-83036fbdcdbb0c5c.arrow


3138 1046


Token indices sequence length is longer than the specified maximum sequence length for this model (943 > 512). Running this sequence through the model will result in indexing errors



Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


392/392 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.0026 - loss: 8.0908 

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


392/392 [==============================] - 54s 138ms/step - sparse_categorical_accuracy: 0.0026 - loss: 8.0908 - val_sparse_categorical_accuracy: 0.0029 - val_loss: 7.3253
Epoch 2/5
392/392 [==============================] - 15s 37ms/step - sparse_categorical_accuracy: 0.0028 - loss: 6.8272 - val_sparse_categorical_accuracy: 0.0030 - val_loss: 6.8818
Epoch 3/5
392/392 [==============================] - 15s 38ms/step - sparse_categorical_accuracy: 0.0031 - loss: 6.5098 - val_sparse_categorical_accuracy: 0.0033 - val_loss: 6.7480
Epoch 4/5
392/392 [==============================] - 15s 37ms/step - sparse_categorical_accuracy: 0.0034 - loss: 6.3287 - val_sparse_categorical_accuracy: 0.0036 - val_loss: 6.6784
Epoch 5/5
392/392 [==============================] - 15s 37ms/step - sparse_categorical_accuracy: 0.0038 - loss: 6.1812 - val_sparse_categorical_accuracy: 0.0037 - val_loss: 6.6354


In [14]:
### Finetune


with strategy.scope():
    config = DistilBertConfig()
    config.attention_type = "performer"
    # Necessary for the classification head created in the model, as CMNLI has 3 labels
    config.num_labels = 3
    config.vocab_size = 21128 # BERT CN Vocab only has 21128
    config.max_position_embeddings = 512 # same as seq_len

    model_f = TFDistilBertForSequenceClassification(config)
    # Change the core model to pretrained version
    model_f.model = model.model
    model_f = WrapperFinetuning(model_f)

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model_f.compile(optimizer=optimizer,
                  steps_per_execution=100,
                  metrics=['sparse_categorical_accuracy'])

batch_size = 8

train_ds, val_ds = get_dataset_f(batch_size, seq_len=512, is_training=True)

model_f.fit(
          train_ds,
          epochs=5,
          validation_data=val_ds,
          )

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-a4cf23710025995b/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-92b47242d2dd7ae8/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.4057 - loss: 1.0600

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


12243/12243 [==============================] - 270s 22ms/step - sparse_categorical_accuracy: 0.4058 - loss: 1.0600 - val_sparse_categorical_accuracy: 0.4472 - val_loss: 1.0427
Epoch 2/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.4676 - loss: 1.0155 - val_sparse_categorical_accuracy: 0.4672 - val_loss: 1.0156
Epoch 3/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.4939 - loss: 0.9873 - val_sparse_categorical_accuracy: 0.4826 - val_loss: 1.0081
Epoch 4/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.5148 - loss: 0.9620 - val_sparse_categorical_accuracy: 0.4783 - val_loss: 1.0078
Epoch 5/5
12243/12243 [==============================] - 232s 19ms/step - sparse_categorical_accuracy: 0.5363 - loss: 0.9344 - val_sparse_categorical_accuracy: 0.4869 - val_loss: 1.0182


In [18]:
### Pretrain Again on dataset to be finetuned


with strategy.scope():
    config = DistilBertConfig()
    config.attention_type = "performer"
    config.vocab_size = 21128 # BERT CN Vocab only has 21128
    config.max_position_embeddings = 512 # same as seq_len

    model_f_pre = TFDistilBertForMaskedLM(config)
    # Change the core model to pretrained version
    model_f_pre = model.model
    model_f_pre = Wrapper(model_f_pre)

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model_f_pre.compile(optimizer=optimizer,
                  steps_per_execution=100,
                  metrics=['sparse_categorical_accuracy'])

batch_size = 8

train_ds, val_ds = get_dataset_f_pre(batch_size, seq_len=512, is_training=True)

model_f_pre.fit(
          train_ds,
          epochs=5,
          validation_data=val_ds,
          )

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-a4cf23710025995b/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-92b47242d2dd7ae8/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)



391783 12426


KeyboardInterrupt: ignored